In [1]:
import pandas as pd
from tqdm import tqdm

import chromadb
from chromadb.config import Settings

In [2]:
data = pd.read_json('bert_vectors_data.json')

data.head()

,num,name,chunks_text,text_vector_bert
0,9263968,Bling Empire S03 E02 Gossip Guy (2022),advertise product brand contact www opensubtit...,"[0.11131072040000001, 0.2290139347, 0.07675495..."
1,9441329,The Low Tone Club (2022),okay four star form part new generation los ag...,"[-0.0255548209, 0.0356444195, 0.14518733320000..."
2,9441329,The Low Tone Club (2022),want put verdi six feet look children let try ...,"[0.0353549197, -0.1301862001, 0.0651464164, 0...."
3,9441329,The Low Tone Club (2022),deal deal say would think yeah continue think ...,"[0.07811786230000001, -0.044406075, -0.1780554..."
4,9441329,The Low Tone Club (2022),many stairs let keep go come roma wrong feel o...,"[0.1022469476, 0.0313826464, -0.0116566606, 0...."


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 310575 entries, 0 to 310574
Data columns (total 4 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   num               310575 non-null  int64 
 1   name              310575 non-null  object
 2   chunks_text       310575 non-null  object
 3   text_vector_bert  310575 non-null  object
dtypes: int64(1), object(3)
memory usage: 11.8+ MB


In [4]:
chroma_client = chromadb.EphemeralClient()

chroma_client.count_collections()

0

In [5]:
client = chromadb.PersistentClient(path="Search_Engine_chromaDB")

collection = client.create_collection(name='Search_Engine',metadata={"hnsw:space": "cosine"})

In [6]:
for i, row in tqdm(data.iterrows(), total=len(data), desc="Processing rows"):
    
    embedding = row['text_vector_bert']
    
    if embedding: 
        
        ids = f"chunk_{i}"  
        
        embedding_list = [float(x) for x in embedding]
        
        collection.add(ids, embeddings = embedding_list, documents = row["chunks_text"], metadatas = {"name": row["name"]})
        
    else:
        
        row['text_vector_bert'] = [0.0] * len(row['text_vector_bert'])

Processing rows: 100%|███████████████████████████████████████████████████████| 310575/310575 [1:37:39<00:00, 53.01it/s]
